In [18]:
### WOOKIEPEDIA SCRAPER ###
import requests
import random
import json
import bs4
import urllib.parse
import re
import os
from collections import defaultdict

# Generated articles are tracked here
TRACKING_ARTICLE_FILE = "../data/generated_articles.log"
BASE_ARTICLE_DIRECTORY = "../data/generated_articles" # Base directory for all saved articles

# --- CATEGORY DEFINITION AND FILTERING SECTIONS ---

# 1. CATEGORIES TO ALWAYS EXCLUDE from being processed/saved, regardless of source
#    These are often meta-categories, real-world, or irrelevant technical tags.
EXCLUDE_CATEGORIES_FOR_ANY_USE = {
    "Real-world people", "Real-world media", "Disambiguation pages", "Dates",
    "Community content", "Wookieepedia", "Wiki", "Articles with unpopulated pronoun parameters", 
    "Articles that use DPL", "Articles with incorrect canonical link",
    "Articles with an inconsistent canonical status",
    "Articles with broken file links", "Articles with dead external links",
    "Articles with information from unknown sources"
}

# 2. CATEGORIES TO AVOID USING AS FOLDER NAMES (too general or internal wiki-maintenance)
#    These categories might be on a page, but are not specific or descriptive or just too general.
AVOID_FOLDER_CATEGORIES = {
    "Articles needing illustration", "Articles with conjectural titles", "Articles with gameplay alternatives",
    "Archiveurl usages with non-Wayback URLs", "Canon articles", "Legends articles", "Canon articles with Legends counterparts"
}
# --- END CATEGORY DEFINITION AND FILTERING SECTIONS ---

# --- ARTICLE PROCESSING FUNCTIONS ---

# 1. Function to get previously generated titles from the tracking file
def get_previously_generated_titles(TRACKING_ARTICLE_FILE):
    """Reads the tracking file and returns a set of titles that have already been generated."""
    if not os.path.exists(TRACKING_ARTICLE_FILE):
        return set()
    with open(TRACKING_ARTICLE_FILE, 'r', encoding='utf-8') as f:
        return set(line.strip() for line in f)

# 2. Function to log a newly generated title
def log_generated_title(title, TRACKING_ARTICLE_FILE):
    """Appends a new title to the tracking file."""
    with open(TRACKING_ARTICLE_FILE, 'a', encoding='utf-8') as f:
        f.write(title + '\n')

# 3. Function to save the article content in a structured, token-efficient format
def save_article_to_file(title, parsed_data, image_urls, category_name="Uncategorized"):
    """Saves the structured, token-efficient article content to a text file in a category-specific folder."""
    
    # Sanitize category_name to be safe for a directory name
    sanitized_category_name = re.sub(r'[\\/*?:"<>|]', "", category_name).strip()
    if not sanitized_category_name: # Fallback if sanitization results in an empty string
        sanitized_category_name = "Uncategorized"

    # Create the category directory if it doesn't exist
    target_directory = os.path.join(BASE_ARTICLE_DIRECTORY, sanitized_category_name)
    os.makedirs(target_directory, exist_ok=True)

    sanitized_title = re.sub(r'[\\/*?:"<>|]', "", title.replace('/', '_'))
    filename = os.path.join(target_directory, f"{sanitized_title}.txt")
    
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(f"Title: {title}\n")

        # Write each content section from the parsed data
        for section, text in parsed_data['content_sections'].items():
            f.write(f"{section}: {text}\n")
        
        # Write the compacted table data
        if parsed_data['infobox_string']:
            f.write(f"Table: {parsed_data['infobox_string']}\n")
            
        # Write the comma-separated appearances
        if parsed_data['appearances']:
            appearances_line = ", ".join(parsed_data['appearances'])
            f.write(f"Appearances: {appearances_line}\n")
            
        # Write the comma-separated image URLs
        if image_urls:
            images_line = ", ".join(image_urls)
            f.write(f"Images: {images_line}\n")
            
    print(f"Article content saved to '{filename}' in AI-friendly format.")

# 4. Function to get a random title from Wookieepedia, checking against the tracking file and defined exclusion categories
def get_random_title(TRACKING_ARTICLE_FILE, max_attempts=30):
    """Gets a random title, checking against the tracking file and defined exclusion categories."""
    url = "https://starwars.fandom.com/api.php"
    generated_titles = get_previously_generated_titles(TRACKING_ARTICLE_FILE)
    print(f"Loaded {len(generated_titles)} previously generated titles.")
    
    for attempt in range(max_attempts):
        print(f"\nAttempt {attempt + 1}/{max_attempts} to find a new random article...")
        params = {"action": "query", "format": "json", "list": "random", "rnnamespace": 0, "rnlimit": 1}
        res = requests.get(url, params=params).json()
        random_title = res['query']['random'][0]['title']
        
        if random_title in generated_titles:
            print(f"'{random_title}' has already been generated. Skipping.")
            continue
        
        try:
            # Fetch categories for filtering
            params = {"action": "query", "format": "json", "titles": random_title, "prop": "categories", "cllimit": "max"}
            category_check_res = requests.get(url, params=params).json()
            page = next(iter(category_check_res["query"]["pages"].values()))
            
            page_categories = set()
            if 'categories' in page:
                page_categories = {cat['title'].replace("Category:", "") for cat in page['categories']}
                
            # Check for exclusion categories (using the centralized EXCLUDE_CATEGORIES_FOR_ANY_USE)
            if not page_categories.isdisjoint(EXCLUDE_CATEGORIES_FOR_ANY_USE):
                print(f"'{random_title}' is in an excluded category ({page_categories.intersection(EXCLUDE_CATEGORIES_FOR_ANY_USE)}). Skipping.")
                continue

            encoded_title = urllib.parse.quote(random_title.replace(' ', '_'))
            print("Found a new, valid random article URL:", f"https://starwars.fandom.com/wiki/{encoded_title}")
            
            return random_title, page_categories
        except requests.exceptions.RequestException as e:
            print(f"API request failed: {e}. Skipping attempt.")
            continue
    return None, set() # Return None and an empty set if no title is found

# 5. Function to get all page data, including images and structured content
def get_article_content(title):
    """Gets all page data and returns it in a structured dictionary."""
    url = "https://starwars.fandom.com/api.php"
    params = {"action": "query", "format": "json", "prop": "images", "titles": title}
    res = requests.get(url, params=params).json()
    page = next(iter(res["query"]["pages"].values()))
    
    parsed_data = get_summary_from_html(title)
    
    return {
        "title": title,
        "parsed_data": parsed_data,
        "images": page.get("images", [])
    }

# 6. Function to filter out images based on specific keywords
# Todo: This functon doesnt work always properly, it should be improved
def filter_images(image_titles):
    exclude_keywords = [
        "logo", "banner", "icon", "question", "cite", "premium", "gotocanon", 
        "gotolegends", "swcustom", "tab-", "onacanonarticle", "onalegendsarticle", 
        "blue-exclamation-mark", "starwars-databank", "food-stub", "bobawhere", "char-stub",
        "falactic_senate", "swtor_mini", "onanoncanonarticle", "swajsmall", "wizardsofthecoast",
        "wiki-shrinkable", "lego", "military-stub", "SWTOR_mini", "kdy", "swinsider", "SWInsider",
        "stub", "planet-stub", "Planet-stub"
    ]
    filtered = []
    for image in image_titles:
        title = image['title'].lower()
        if not any(keyword in title for keyword in exclude_keywords):
            filtered.append(image)
    return filtered

# 7. Function to get image URLs from the filtered image titles
def get_image_urls(image_titles):
    urls = []
    for image in image_titles:
        title = image['title']
        url = "https://starwars.fandom.com/api.php"
        params = {"action": "query", "format": "json", "titles": title, "prop": "imageinfo", "iiprop": "url"}
        res = requests.get(url, params=params).json()
        page = next(iter(res['query']['pages'].values()))
        if 'imageinfo' in page:
            urls.append(page['imageinfo'][0]['url'])
    return urls

# 8. Function to scrape a Wookieepedia page and return structured content
def get_summary_from_html(title):
    """
    Scrapes a Wookieepedia page and returns a structured dictionary of its content.
    """
    url = f"https://starwars.fandom.com/wiki/{urllib.parse.quote(title.replace(' ', '_'))}"
    
    try:
        res = requests.get(url)
        res.raise_for_status()
        soup = bs4.BeautifulSoup(res.text, 'html.parser')
        
        # --- Data Extraction into Intermediate Structures ---
        infobox_data = defaultdict(list)
        infobox = soup.find('aside', class_='portable-infobox')
        if infobox:
            current_section = infobox.find('h2', class_='pi-title').get_text(strip=True) if infobox.find('h2', 'pi-title') else "General"
            for item in infobox.find_all(['h2', 'div'], class_=['pi-header', 'pi-data']):
                if 'pi-header' in item.get('class', []):
                    current_section = item.get_text(strip=True)
                elif 'pi-data' in item.get('class', []):
                    label_el = item.find('h3', class_='pi-data-label')
                    value_el = item.find('div', class_='pi-data-value')
                    if label_el and value_el:
                        for sup in value_el.find_all('sup', class_='reference'): sup.decompose()
                        key = label_el.get_text(strip=True)
                        value = ' '.join(value_el.get_text(separator=' ', strip=True).split())
                        infobox_data[current_section].append(f"{key}: {value}")
            infobox.decompose()

        # --- Content Section Parsing ---
        content_sections = defaultdict(list)
        appearances = []
        current_section_key = 'Main' # For text before the first header
        content_div = soup.find('div', class_='mw-parser-output')
        if content_div:
            parsing_appearances = False
            for element in content_div.find_all(['h2', 'h3', 'p', 'div', 'ul'], recursive=False):
                if element.name in ['h2', 'h3']:
                    headline_span = element.find('span', class_='mw-headline')
                    if not headline_span: continue
                    headline_text = headline_span.get_text(strip=True)
                    stop_keywords = ["Sources", "Notes and references", "See also", "External links"]
                    if any(keyword in headline_text for keyword in stop_keywords): break
                    parsing_appearances = "Appearances" in headline_text
                    if not parsing_appearances:
                        current_section_key = headline_text
                elif parsing_appearances and element.name in ['div', 'ul']:
                    for li in element.find_all('li'):
                        raw_text = li.get_text(strip=True)
                        cleaned_text = re.sub(r'\s*\([^)]*\)', '', raw_text).strip()
                        if cleaned_text: appearances.append(cleaned_text)
                elif not parsing_appearances and element.name == 'p':
                    for sup in element.find_all('sup', class_='reference'): sup.decompose()
                    text = element.get_text(separator=' ', strip=True)
                    if text: content_sections[current_section_key].append(text)

        # --- Final Formatting into Token-Efficient Strings ---
        final_sections = {key: " ".join(value) for key, value in content_sections.items()}
        
        infobox_string_parts = []
        for section, kv_pairs in infobox_data.items():
            pairs_str = "; ".join(kv_pairs)
            infobox_string_parts.append(f"{section} | {pairs_str}")
        infobox_final_str = " | ".join(infobox_string_parts)
        
        return {
            "infobox_string": infobox_final_str,
            "content_sections": final_sections,
            "appearances": appearances
        }
    except Exception as e:
        return {"infobox_string": "", "content_sections": {"Error": str(e)}, "appearances": []}
# --- END ARTICLE PROCESSING FUNCTIONS ---

# --- Main Execution ---
if __name__ == "__main__":
    
    # Create the base saving directory if it doesn't exist
    os.makedirs(BASE_ARTICLE_DIRECTORY, exist_ok=True)
    
    # Always try to get a random title
    title, categories = get_random_title(TRACKING_ARTICLE_FILE)
    
    if title:
        page_data = get_article_content(title)
        
        if "Error" in page_data["parsed_data"]["content_sections"]:
            print(f"Could not process '{title}'. Reason: {page_data['parsed_data']['content_sections']['Error']}")
        else:
            filtered_images = filter_images(page_data["images"])
            image_urls = get_image_urls(filtered_images)
            
            chosen_category_for_folder = "Uncategorized" # Default fallback

            # Combine all exclusion sets for checking against potential folder names
            all_avoid_categories_for_folder_name = AVOID_FOLDER_CATEGORIES.union(EXCLUDE_CATEGORIES_FOR_ANY_USE)
            
            # Convert categories set to a sorted list to ensure deterministic "first" tag
            sorted_page_categories = sorted(list(categories))
            
            # Iterate through the page's categories and pick the first suitable one for the folder name
            for cat in sorted_page_categories:
                # Check if the category is suitable (not in any exclusion/avoid list, and not a generic keyword)
                if cat not in all_avoid_categories_for_folder_name and \
                   not any(keyword in cat.lower() for keyword in ["real-world", "disambiguation", "date", "fictional"]):
                    chosen_category_for_folder = cat
                    break # Use this category and stop searching

            # Save the structured data to the file
            save_article_to_file(
                page_data["title"], 
                page_data["parsed_data"], 
                image_urls,
                category_name=chosen_category_for_folder # Pass the determined category
            )
            
            log_generated_title(page_data["title"], TRACKING_ARTICLE_FILE)
            print(f"\nProcess complete. '{title}' has been saved and logged in the '{chosen_category_for_folder}' folder.")
    else:
        print("\nCould not find a new article to process after multiple attempts.")

Loaded 12 previously generated titles.

Attempt 1/30 to find a new random article...
Found a new, valid random article URL: https://starwars.fandom.com/wiki/Kilogram
Article content saved to '../data/generated_articles\Science stubs\Kilogram.txt' in AI-friendly format.

Process complete. 'Kilogram' has been saved and logged in the 'Science stubs' folder.


In [19]:
### AI STUFF ###
import os
import re
from dotenv import load_dotenv
import google.generativeai as genai
from google.cloud import texttospeech # Import WaveNet library

# --- Gemini Configuration ---
load_dotenv()
gemini_api_key = os.getenv("GEMINI_API_KEY")
if gemini_api_key is None:
    raise ValueError("GEMINI_API_KEY not found in environment variables. "
                     "Please set it in a .env file or as an environment variable.")

# --- Google Cloud Text-to-Speech Configuration ---
# Get the path to the JSON key from .env
google_app_credentials_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
if google_app_credentials_path is None:
    raise ValueError("GOOGLE_APPLICATION_CREDENTIALS not found in environment variables. "
                     "Please set it in a .env file or as an environment variable.")

# IMPORTANT: Explicitly set the environment variable for Google Cloud libraries
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = google_app_credentials_path
print(f"Set GOOGLE_APPLICATION_CREDENTIALS to: {os.environ['GOOGLE_APPLICATION_CREDENTIALS']}")

# --- Initialize Gemini API client ---
genai.configure(api_key=gemini_api_key)
model = genai.GenerativeModel('gemini-1.5-flash') # Recommended for quick text tasks

# --- Initialize Google Cloud Text-to-Speech Client ---
try:
    tts_client = texttospeech.TextToSpeechClient()
    print("Google Cloud Text-to-Speech client initialized.")
except Exception as e:
    # Adding a check here to ensure the file exists at the set path
    if not os.path.exists(google_app_credentials_path):
        raise RuntimeError(f"Failed to initialize Google Cloud Text-to-Speech client. "
                           f"The JSON key file was NOT found at the specified path: {google_app_credentials_path}. "
                           f"Error: {e}")
    else:
        raise RuntimeError(f"Failed to initialize Google Cloud Text-to-Speech client. "
                           f"Ensure GOOGLE_APPLICATION_CREDENTIALS is set correctly and the JSON file exists: {e}")


# --- File Paths ---
BASE_ARTICLE_DIRECTORY = "../data/generated_articles"
TRACKING_ARTICLE_FILE = "../data/generated_articles.log"

# Path for saving the generated text
GENERATED_TEXT_DIRECTORY = "../data/generated_text"

# Voice generation specific paths
VOICE_SAVE_DIRECTORY = "../data/generated_audio"
TRACKING_VOICE_FILE = "../data/generated_voice.log" # Tracks which articles have had voice generated

# --- Helper Functions ---

# 1. Function to get previously processed voice titles
def get_processed_voice_titles(tracking_file):
    """Reads the voice tracking file and returns a set of titles already processed for voice."""
    if not os.path.exists(tracking_file):
        return set()
    with open(tracking_file, 'r', encoding='utf-8') as f:
        return set(line.strip() for line in f)

# 2. Function to log a newly processed voice title
def log_processed_voice_title(title, tracking_file):
    """Appends a new title to the voice tracking file."""
    with open(tracking_file, 'a', encoding='utf-8') as f:
        f.write(title + '\n')

# 3. Function to parse a saved article file into its components
def parse_article_file(filepath):
    """
    Parses a saved article file into its components.
    Returns (title, main_content_for_gemini, images_part)
    """
    title = ""
    main_content_lines = []
    table_content_lines = []
    appearances_content_lines = []
    images_part = ""

    current_section = None
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line.startswith("Title:"):
                title = line[len("Title:"):].strip()
                current_section = None # Reset after title
            elif line.startswith("Main:"):
                current_section = "Main"
                main_content_lines.append(line[len("Main:"):].strip())
            elif line.startswith("Table:"):
                current_section = "Table"
                table_content_lines.append(line[len("Table:"):].strip())
            elif line.startswith("Appearances:"):
                current_section = "Appearances"
                appearances_content_lines.append(line[len("Appearances:"):].strip())
            elif line.startswith("Images:"):
                current_section = "Images"
                images_part = line[len("Images:"):].strip()
            elif current_section: # Append to current section if continuation line
                if current_section == "Main":
                    main_content_lines.append(line)
                elif current_section == "Table":
                    table_content_lines.append(line)
                elif current_section == "Appearances":
                    appearances_content_lines.append(line)
    
    # Concatenate content for Gemini
    gemini_input_parts = []
    if title:
        gemini_input_parts.append(f"Title: {title}")
    if main_content_lines:
        gemini_input_parts.append(f"Main: {' '.join(main_content_lines)}")
    if table_content_lines:
        gemini_input_parts.append(f"Table: {' '.join(table_content_lines)}")
    if appearances_content_lines:
        gemini_input_parts.append(f"Appearances: {' '.join(appearances_content_lines)}")
        
    content_for_gemini = "\n".join(gemini_input_parts)

    return title, content_for_gemini, images_part

# 4. Function to rephrase text using Gemini API
def rephrase_with_gemini(text_to_rephrase):
    """
    Sends text to Gemini API for rephrasing.
    """
    prompt = f"Rephrase the following Star Wars Wookieepedia article content in an engaging, concise, and informative way, suitable for a narration. Focus on the core facts and avoid phrases like 'this article describes' or 'the content above':\n\n{text_to_rephrase}"
    
    try:
        response = model.generate_content(prompt)
        if response.parts and response.parts[0].text:
            return response.parts[0].text
        else:
            print(f"Gemini response was empty or malformed for text: {text_to_rephrase[:100]}...")
            return None
    except Exception as e:
        print(f"Error calling Gemini API: {e}")
        return None

# 5. Function to save the rephrased text for subtitles
def save_rephrased_text_for_subtitles(rephrased_text, original_title, category_folder):
    """
    Saves the rephrased text to a new file in the generated_text directory,
    under a category-specific subfolder.
    """
    # Sanitize category_folder to be safe for a directory name
    sanitized_category_folder = re.sub(r'[\\/*?:"<>|]', "", category_folder).strip()
    if not sanitized_category_folder:
        sanitized_category_folder = "Uncategorized_Text"

    target_directory = os.path.join(GENERATED_TEXT_DIRECTORY, sanitized_category_folder)
    os.makedirs(target_directory, exist_ok=True) # Ensure the directory exists

    sanitized_title = re.sub(r'[\\/*?:"<>|]', "", original_title.replace('/', '_'))
    filename = os.path.join(target_directory, f"{sanitized_title}_rephrased.txt")

    try:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(rephrased_text)
        print(f"Rephrased text saved to '{filename}'.")
        return True
    except Exception as e:
        print(f"Error saving rephrased text for '{original_title}': {e}")
        return False

# 6. Function to generate audio using Google Cloud WaveNet
def generate_wavenet_audio(text_to_convert, original_title, category_folder):
    """
    Converts text to speech using Google Cloud WaveNet and saves it as an MP3 file.
    """
    # Sanitize category_folder for directory name
    sanitized_category_folder = re.sub(r'[\\/*?:"<>|]', "", category_folder).strip()
    if not sanitized_category_folder:
        sanitized_category_folder = "Uncategorized_Audio"

    target_directory = os.path.join(VOICE_SAVE_DIRECTORY, sanitized_category_folder)
    os.makedirs(target_directory, exist_ok=True)

    sanitized_title = re.sub(r'[\\/*?:"<>|]', "", original_title.replace('/', '_'))
    filename = os.path.join(target_directory, f"{sanitized_title}.mp3")

    synthesis_input = texttospeech.SynthesisInput(text=text_to_convert)

    # Select the voice parameters. You can change these!
    # Available voices: https://cloud.google.com/text-to-speech/docs/voices
    # 'en-US-Wavenet-D' is a common male voice. 'en-US-Wavenet-C' is a common female voice.
    voice = texttospeech.VoiceSelectionParams(
        language_code="en-US",
        name="en-US-Wavenet-D", # Choose your preferred WaveNet voice
        ssml_gender=texttospeech.SsmlVoiceGender.MALE # Match gender to voice name
    )

    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3,
        speaking_rate=1.0, # Adjust speed if desired (0.25 to 4.0)
        pitch=0.0 # Adjust pitch if desired (-20.0 to 20.0)
    )

    try:
        response = tts_client.synthesize_speech(
            input=synthesis_input, voice=voice, audio_config=audio_config
        )

        with open(filename, "wb") as out:
            out.write(response.audio_content)
        print(f"Audio content saved to '{filename}'.")
        return True
    except Exception as e:
        print(f"Error generating WaveNet audio for '{original_title}': {e}")
        return False

# --- Main Processing Logic ---
if __name__ == "__main__":
    # Ensure all base output directories exist
    os.makedirs(GENERATED_TEXT_DIRECTORY, exist_ok=True)
    os.makedirs(VOICE_SAVE_DIRECTORY, exist_ok=True)

    processed_voice_titles = get_processed_voice_titles(TRACKING_VOICE_FILE)
    print(f"Loaded {len(processed_voice_titles)} previously generated audio titles.")

    for root, dirs, files in os.walk(BASE_ARTICLE_DIRECTORY):
        category_folder = os.path.basename(root) 
        
        # Skip the base directory itself if it's not a category folder
        if category_folder == os.path.basename(BASE_ARTICLE_DIRECTORY) and root != BASE_ARTICLE_DIRECTORY:
             continue 

        for filename in files:
            if filename.endswith(".txt"):
                filepath = os.path.join(root, filename)
                print(f"\nProcessing file: {filepath}")

                original_title, content_for_gemini, images_part = parse_article_file(filepath)

                if not original_title:
                    print(f"Could not parse title from {filename}. Skipping.")
                    continue

                if original_title in processed_voice_titles:
                    print(f"'{original_title}' has already had audio generated. Skipping.")
                    continue
                
                if not content_for_gemini.strip():
                    print(f"No substantial text content found for Gemini in '{original_title}'. Skipping.")
                    continue

                print(f"Sending to Gemini for rephrasing: {original_title}")
                rephrased_text = rephrase_with_gemini(content_for_gemini)

                if rephrased_text:
                    # Save the rephrased text for subtitles
                    text_saved = save_rephrased_text_for_subtitles(rephrased_text, original_title, category_folder)
                    
                    if text_saved:
                        print(f"Successfully rephrased '{original_title}'. Now generating audio...")
                        audio_generated = generate_wavenet_audio(rephrased_text, original_title, category_folder)
                        
                        if audio_generated:
                            log_processed_voice_title(original_title, TRACKING_VOICE_FILE)
                            print(f"Audio generation and logging complete for '{original_title}'.")
                        else:
                            print(f"Audio generation failed for '{original_title}'. Not logging.")
                    else:
                        print(f"Failed to save rephrased text for '{original_title}'. Skipping audio generation.")
                else:
                    print(f"Failed to get rephrased text for '{original_title}'. Skipping text and audio generation.")
    
    print("\n--- Processing complete. ---")

Set GOOGLE_APPLICATION_CREDENTIALS to: C:\egyetem\github-ml\holocron-generator\ethereal-smoke-467209-n7-2fa1b578920d.json
Google Cloud Text-to-Speech client initialized.
Loaded 12 previously generated audio titles.

Processing file: ../data/generated_articles\Arboreal planets\Mulita.txt
'Mulita' has already had audio generated. Skipping.

Processing file: ../data/generated_articles\Arms manufacturing companies\Antrech Arms.txt
'Antrech Arms' has already had audio generated. Skipping.

Processing file: ../data/generated_articles\Battles in 19 BBY\Mission to Batuu.txt
'Mission to Batuu' has already had audio generated. Skipping.

Processing file: ../data/generated_articles\Character stubs\Eos Morne.txt
'Eos Morne' has already had audio generated. Skipping.

Processing file: ../data/generated_articles\Character stubs\Unidentified Second Prince.txt
Sending to Gemini for rephrasing: Unidentified Second Prince
Rephrased text saved to '../data/generated_text\Character stubs\Unidentified Secon

In [20]:
### TRANSCRIPT GENERATION USING FORCE ALIGNMENT (WITH DEBUGGING) ###
import os
from forcealign import ForceAlign
import re # For sanitizing filenames

# Directory where AI generated audio files are stored
GENERATED_AUDIO_DIRECTORY = "../data/generated_audio"

# Directory where AI generated voices transcript files are stored
GENERATED_TEXT_DIRECTORY = "../data/generated_text"

# Directory where the generated SRT transcripts are saved
GENERATED_TRANSCRIPT_DIRECTORY = "../data/generated_transcripts"

# Tracking file for already created shorts
TRACKING_TRANSCRIPT_FILE = "../data/generated_transcripts.log"

# Define the suffix that might be present in text files but not audio
TEXT_FILE_SUFFIX = "_rephrased.txt"

def format_time_for_srt(seconds):
    """Converts seconds to SRT time format: HH:MM:SS,mmm"""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds_remainder = seconds % 60
    milliseconds = int((seconds_remainder - int(seconds_remainder)) * 1000)
    return f"{hours:02d}:{minutes:02d}:{int(seconds_remainder):02d},{milliseconds:03d}"

def create_srt_from_alignment(words_with_timestamps, output_filepath):
    """
    Creates an SRT file from the ForceAlign output.
    This version groups words into sensible subtitle lines (e.g., up to 3-4 words or until a short pause).
    For perfect, word-per-word, you'd modify the grouping logic.
    """
    srt_entries = []
    current_line_words = []
    current_line_start_time = None
    sequence_number = 1
    
    # Heuristic for grouping: a new line after a pause or every few words
    WORD_GROUP_LIMIT = 5 # Max words per line
    PAUSE_THRESHOLD = 0.3 # seconds, for detecting a natural break

    print(f"\n--- Debugging SRT Creation for {os.path.basename(output_filepath)} ---")
    print(f"Total words from alignment: {len(words_with_timestamps)}")

    for i, word_info in enumerate(words_with_timestamps):
        word = word_info.word
        start_time = word_info.time_start
        end_time = word_info.time_end

        # Debug print for each word's timestamp
        print(f"  Word {i+1}: '{word}' (Start: {start_time:.3f}, End: {end_time:.3f})")

        if not current_line_words:
            current_line_start_time = start_time
        
        current_line_words.append(word)

        is_last_word = (i == len(words_with_timestamps) - 1)
        next_word_info = words_with_timestamps[i+1] if not is_last_word else None
        
        should_break = False
        if is_last_word:
            should_break = True
            print("    Break reason: Last word.")
        elif len(current_line_words) >= WORD_GROUP_LIMIT:
            should_break = True
            print(f"    Break reason: Word group limit ({WORD_GROUP_LIMIT}) reached.")
        elif next_word_info and (next_word_info.time_start - end_time > PAUSE_THRESHOLD):
            should_break = True
            print(f"    Break reason: Pause detected ({(next_word_info.time_start - end_time):.3f}s > {PAUSE_THRESHOLD}s).")
        elif re.search(r'[.!?]', word): # Check for punctuation
            should_break = True
            print("    Break reason: Punctuation detected.")

        if should_break:
            line_text = " ".join(current_line_words)
            line_end_time = end_time

            srt_entries.append(f"{sequence_number}")
            srt_entries.append(f"{format_time_for_srt(current_line_start_time)} --> {format_time_for_srt(line_end_time)}")
            srt_entries.append(line_text)
            srt_entries.append("") # Blank line separator for SRT

            print(f"  --- SRT Entry {sequence_number} ---")
            print(f"    Time: {format_time_for_srt(current_line_start_time)} --> {format_time_for_srt(line_end_time)}")
            print(f"    Text: \"{line_text}\"")
            
            current_line_words = []
            current_line_start_time = None
            sequence_number += 1

    # Ensure output directory exists
    os.makedirs(os.path.dirname(output_filepath), exist_ok=True)
    
    with open(output_filepath, "w", encoding="utf-8") as f:
        f.write("\n".join(srt_entries))
    print(f"\nGenerated SRT: {output_filepath}")


def get_processed_log():
    """Reads the log file to get a set of already processed file identifiers."""
    if not os.path.exists(TRACKING_TRANSCRIPT_FILE):
        return set()
    with open(TRACKING_TRANSCRIPT_FILE, "r", encoding="utf-8") as f:
        return set(line.strip() for line in f)

def add_to_processed_log(identifier):
    """Adds a file identifier to the log file."""
    with open(TRACKING_TRANSCRIPT_FILE, "a", encoding="utf-8") as f:
        f.write(f"{identifier}\n")

def process_audio_text_pairs():
    processed_files = get_processed_log()

    for root, _, files in os.walk(GENERATED_AUDIO_DIRECTORY):
        for audio_file_name in files:
            if audio_file_name.endswith((".mp3", ".wav")):
                relative_path = os.path.relpath(root, GENERATED_AUDIO_DIRECTORY)
                base_name = os.path.splitext(audio_file_name)[0]
                audio_file_path = os.path.join(root, audio_file_name)
                text_file_path = os.path.join(GENERATED_TEXT_DIRECTORY, relative_path, f"{base_name}{TEXT_FILE_SUFFIX}")
                file_identifier = os.path.relpath(audio_file_path, GENERATED_AUDIO_DIRECTORY)

                if file_identifier in processed_files:
                    print(f"Skipping already processed: {file_identifier}")
                    continue
                
                if not os.path.exists(text_file_path):
                    print(f"Warning: Corresponding text file not found for audio '{audio_file_path}'. Expected '{text_file_path}'")
                    continue
                
                print(f"\nProcessing: Audio='{audio_file_path}', Text='{text_file_path}'")

                try:
                    with open(text_file_path, "r", encoding="utf-8") as f:
                        transcript = f.read().strip()

                    if not transcript:
                        print(f"Skipping empty transcript: {text_file_path}")
                        add_to_processed_log(file_identifier)
                        continue
                    
                    aligner = ForceAlign(audio_file=audio_file_path, transcript=transcript)
                    words = aligner.inference()

                    output_srt_directory = os.path.join(GENERATED_TRANSCRIPT_DIRECTORY, relative_path)
                    os.makedirs(output_srt_directory, exist_ok=True)
                    srt_file_name = f"{base_name}.srt"
                    output_srt_path = os.path.join(output_srt_directory, srt_file_name)

                    create_srt_from_alignment(words, output_srt_path)
                    add_to_processed_log(file_identifier)

                except Exception as e:
                    print(f"Error processing {file_identifier}: {e}")

if __name__ == "__main__":
    os.makedirs(GENERATED_AUDIO_DIRECTORY, exist_ok=True)
    os.makedirs(GENERATED_TEXT_DIRECTORY, exist_ok=True)
    os.makedirs(GENERATED_TRANSCRIPT_DIRECTORY, exist_ok=True)

    print("Starting forced alignment process...")
    process_audio_text_pairs()
    print("Forced alignment process complete.")

Starting forced alignment process...
Skipping already processed: Arboreal planets\Mulita.mp3
Skipping already processed: Arboreal planets\Mulita.wav
Skipping already processed: Arms manufacturing companies\Antrech Arms.mp3
Skipping already processed: Arms manufacturing companies\Antrech Arms.wav
Skipping already processed: Battles in 19 BBY\Mission to Batuu.mp3
Skipping already processed: Battles in 19 BBY\Mission to Batuu.wav
Skipping already processed: Character stubs\Eos Morne.mp3
Skipping already processed: Character stubs\Eos Morne.wav

Processing: Audio='../data/generated_audio\Character stubs\Unidentified Second Prince.mp3', Text='../data/generated_text\Character stubs\Unidentified Second Prince_rephrased.txt'

--- Debugging SRT Creation for Unidentified Second Prince.srt ---
Total words from alignment: 63
  Word 1: 'IN' (Start: 0.000, End: 0.220)
  Word 2: 'THE' (Start: 0.240, End: 0.340)
  Word 3: 'EMPIRES' (Start: 0.440, End: 0.880)
  Word 4: 'TWILIGHT' (Start: 0.941, End: 1.

In [21]:
### VIDEO GENERATION SCRIPT WITH TRANSCRIPT (WITH DEBUGGING) ###
import os
import re
import random
import requests
from moviepy.editor import VideoFileClip, AudioFileClip, ImageClip, CompositeVideoClip
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import glob
from urllib.parse import urlparse

# --- Configuration ---

# Main Minecraft parkour footage path
MINECRAFT_FOOTAGE_PATH = "../data/minecraft_footage/minecraft01.mp4"

# Directory where AI generated audio files are stored
GENERATED_AUDIO_DIRECTORY = "../data/generated_audio"

# Directory where AI generated article text files are stored (assuming same structure as audio)
GENERATED_ARTICLES_DIRECTORY = "../data/generated_articles"

# Directory where the generated SRT transcripts are saved (new)
GENERATED_TRANSCRIPT_DIRECTORY = "../data/generated_transcripts"

# Directory where the final YouTube Shorts will be saved
GENERATED_SHORTS_DIRECTORY = "../data/generated_shorts"

# Temporary directory for downloaded images
TEMP_IMAGE_DIRECTORY = "../data/temp_images"

# Tracking file for already created shorts
TRACKING_SHORTS_FILE = "../data/generated_shorts.log"

# Target resolution for YouTube Shorts (e.g., 1080x1920 for 9:16 aspect ratio)
TARGET_WIDTH = 1080
TARGET_HEIGHT = 1920 # For 9:16 aspect ratio

# --- Helper Functions for Tracking ---

# 1. Function to get previously processed shorts titles
def get_processed_shorts_titles(tracking_file):
    """Reads the shorts tracking file and returns a set of titles that have already had shorts created."""
    if not os.path.exists(tracking_file):
        return set()
    with open(tracking_file, 'r', encoding='utf-8') as f:
        return set(line.strip() for line in f)

# 2. Function to log a newly processed short title
def log_processed_short_title(title, tracking_file):
    """Appends a new title to the shorts tracking file."""
    with open(tracking_file, 'a', encoding='utf-8') as f:
        f.write(title + '\n')

# --- Image Handling Functions ---

# 1. Function to extract image URLs from the article content
def extract_image_urls_from_article(article_content):
    """Extracts image URLs from the 'Images:' line in the article content."""
    urls = []
    match = re.search(r"Images:\s*(.*)", article_content, re.IGNORECASE)
    if match:
        url_string = match.group(1).strip()
        potential_urls = url_string.split()
        for url in potential_urls:
            if url.startswith("http://") or url.startswith("https://"):
                urls.append(url)
    return urls

# 2. Function to download an image from a URL
def download_image(url, destination_folder):
    """Downloads an image from a URL to a specified folder and returns its local path."""
    os.makedirs(destination_folder, exist_ok=True)
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()

        parsed_url = urlparse(url)
        filename = os.path.basename(parsed_url.path)
        if not filename:
            filename = "downloaded_image.png"

        base, ext = os.path.splitext(filename)
        if ext.lower() == '.svg':
            print(f"Skipping SVG image {url} as direct SVG support is not enabled.")
            return None # Skip SVG for now

        local_filepath = os.path.join(destination_folder, f"{base}_{random.randint(1000,9999)}{ext}")

        with open(local_filepath, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"Downloaded image from {url} to {local_filepath}")
        return local_filepath
    except requests.exceptions.RequestException as e:
        print(f"Error downloading image from {url}: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during image download from {url}: {e}")
        return None
    
# 3. Function to create a temporary image from a URL
def clean_temp_images(directory):
    """Removes all files from the temporary image directory."""
    if os.path.exists(directory):
        for file in os.listdir(directory):
            file_path = os.path.join(directory, file)
            try:
                if os.path.isfile(file_path):
                    os.unlink(file_path)
            except Exception as e:
                print(f"Error deleting temporary file {file_path}: {e}")
        print(f"Cleaned temporary image directory: {directory}")

# --- Subtitle Handling Functions ---

def parse_srt(srt_filepath):
    """Parses an SRT file and returns a list of dictionaries, each containing
    'start', 'end', and 'text' for a subtitle entry."""
    
    subtitles = []
    current_block = []
    
    if not os.path.exists(srt_filepath):
        print(f"SRT file not found: {srt_filepath}")
        return subtitles

    with open(srt_filepath, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line: # Empty line indicates end of a subtitle block
                if current_block:
                    try:
                        sequence_number = int(current_block[0])
                        time_str = current_block[1]
                        start_time_str, end_time_str = time_str.split(' --> ')
                        
                        def srt_time_to_seconds(time_str_val):
                            parts = time_str_val.replace(',', '.').split(':')
                            if len(parts) == 3:
                                h, m, s = float(parts[0]), float(parts[1]), float(parts[2])
                                return h * 3600 + m * 60 + s
                            return 0

                        start_seconds = srt_time_to_seconds(start_time_str)
                        end_seconds = srt_time_to_seconds(end_time_str)
                        text = " ".join(current_block[2:])
                        
                        subtitles.append({
                            'start': start_seconds,
                            'end': end_seconds,
                            'text': text
                        })
                    except (ValueError, IndexError) as e:
                        print(f"Warning: Could not parse SRT block: {current_block}. Error: {e}")
                    current_block = []
            else:
                current_block.append(line)
        
        if current_block:
            try:
                sequence_number = int(current_block[0])
                time_str = current_block[1]
                start_time_str, end_time_str = time_str.split(' --> ')
                
                def srt_time_to_seconds(time_str_val):
                    parts = time_str_val.replace(',', '.').split(':')
                    if len(parts) == 3:
                        h, m, s = float(parts[0]), float(parts[1]), float(parts[2])
                        return h * 3600 + m * 60 + s
                    return 0

                start_seconds = srt_time_to_seconds(start_time_str)
                end_seconds = srt_time_to_seconds(end_time_str)
                text = " ".join(current_block[2:])
                
                subtitles.append({
                    'start': start_seconds,
                    'end': end_seconds,
                    'text': text
                })
            except (ValueError, IndexError) as e:
                print(f"Warning: Could not parse final SRT block: {current_block}. Error: {e}")
    
    return subtitles

# --- Video Processing Function ---

# 1. Function to wrap text into multiple lines dynamically based on width
def dynamic_wrap_text(text, font, max_width):
    """
    Dynamically wraps text into multiple lines to fit within a max_width,
    returning a list of lines.
    """
    lines = []
    if not text:
        return lines

    words = text.split()
    current_line = []
    
    dummy_draw = ImageDraw.Draw(Image.new("RGB", (1,1)))

    for word in words:
        test_line = " ".join(current_line + [word])
        
        try:
            bbox = dummy_draw.textbbox((0,0), test_line, font=font)
            test_line_width = bbox[2] - bbox[0]
        except AttributeError:
            test_line_width, _ = dummy_draw.textsize(test_line, font=font)

        if test_line_width <= max_width:
            current_line.append(word)
        else:
            if not current_line:
                lines.append(word)
                current_line = []
            else:
                lines.append(" ".join(current_line))
                current_line = [word]
    
    if current_line:
        lines.append(" ".join(current_line))

    return lines

# 2. Function to create a YouTube Short from Minecraft footage and AI audio
def create_youtube_short(
    audio_filepath,
    minecraft_footage_path,
    output_base_dir,
    article_title,
    category_folder,
    image_urls,
    srt_filepath,
    target_width=1080,
    target_height=1920
):
    """
    Creates a YouTube Short by combining a random segment of Minecraft footage with AI audio,
    cropping it to 9:16 aspect ratio, and saving it. Includes title text, image overlay, and subtitles.
    """
    clean_temp_images(TEMP_IMAGE_DIRECTORY)

    subtitles = parse_srt(srt_filepath)
    print(f"Loaded {len(subtitles)} subtitle entries from: {srt_filepath}")

    try:
        audio_clip = AudioFileClip(audio_filepath)
        audio_duration = audio_clip.duration
        
        print(f"Loaded audio: {os.path.basename(audio_filepath)} (Duration: {audio_duration:.2f}s)")

        main_video_clip = VideoFileClip(minecraft_footage_path)
        full_video_duration = main_video_clip.duration
        
        print(f"Loaded main video: {os.path.basename(minecraft_footage_path)} (Duration: {full_video_duration:.2f}s)")

        max_start_time_based_on_80_percent = full_video_duration * 0.8
        max_possible_start_time_for_audio_fit = full_video_duration - audio_duration

        actual_max_start_time = min(max_start_time_based_on_80_percent, max_possible_start_time_for_audio_fit)

        if actual_max_start_time < 0:
            print(f"Warning: Audio duration ({audio_duration:.2f}s) is longer than main video ({full_video_duration:.2f}s). Skipping {article_title}.")
            main_video_clip.close()
            audio_clip.close()
            return False
            
        if actual_max_start_time == 0 and full_video_duration > audio_duration:
            random_start_time = 0
            print(f"Video is short, starting at 0s.")
        else:
            random_start_time = random.uniform(0, actual_max_start_time)

        end_time = random_start_time + audio_duration
        
        if end_time > full_video_duration + 0.01:
             end_time = full_video_duration
             if (end_time - random_start_time) < audio_duration * 0.95:
                 print(f"Warning: Adjusted video end time. Clip might be slightly shorter than audio for {article_title}.")
                 
        print(f"Selected video segment: {random_start_time:.2f}s to {end_time:.2f}s (Clip duration: {(end_time - random_start_time):.2f}s)")

        video_clip = main_video_clip.subclip(random_start_time, end_time)
        video_clip = video_clip.set_audio(None)
        video_clip = video_clip.set_audio(audio_clip)

        original_width, original_height = video_clip.size
        target_aspect_ratio = target_width / target_height 
        original_aspect_ratio = original_width / original_height

        if original_aspect_ratio > target_aspect_ratio:
            new_width = int(original_height * target_aspect_ratio)
            x_center = original_width / 2
            x1 = int(x_center - new_width / 2)
            y1 = 0
            cropped_clip = video_clip.crop(x1=x1, y1=y1, width=new_width, height=original_height)
            print(f"Cropping width from {original_width} to {new_width} to fit 9:16.")
        else:
            new_height = int(original_width / target_aspect_ratio)
            y_center = original_height / 2
            y1 = int(y_center - new_height / 2)
            x1 = 0
            cropped_clip = video_clip.crop(x1=x1, y1=y1, width=original_width, height=new_height)
            print(f"Cropping height from {original_height} to {new_height} to fit 9:16.")

        final_clip = cropped_clip.resize(newsize=(target_width, target_height))
        print(f"Resized to final resolution: {final_clip.size[0]}x{final_clip.size[1]}")

        # --- Image Overlay Logic ---
        downloaded_image_paths = []
        if image_urls:
            print(f"Attempting to download {len(image_urls)} images for '{article_title}'...")
            for url in image_urls:
                path = download_image(url, TEMP_IMAGE_DIRECTORY)
                if path:
                    downloaded_image_paths.append(path)
            
            if downloaded_image_paths:
                image_duration_per_clip = audio_duration / len(downloaded_image_paths)
                print(f"Images will switch every {image_duration_per_clip:.2f} seconds.")
            else:
                print("No images successfully downloaded for overlay.")
        else:
            print("No image URLs found for overlay.")

        def draw_elements_on_frame(get_frame, t_in_clip):
            image_array = get_frame(t_in_clip)
            # Convert to RGBA for consistent handling of potential transparent overlays
            img_pil = Image.fromarray(image_array.astype('uint8'), 'RGB').convert("RGBA")
            draw = ImageDraw.Draw(img_pil)

            t_actual_transcript = t_in_clip
            
            if int(t_in_clip * 10) % 10 == 0:
                print(f"  Frame at t_in_clip: {t_in_clip:.2f}s (Time for subtitle lookup: {t_actual_transcript:.2f}s)")

            # --- TITLE DRAWING LOGIC ---
            font_path = "../data/fonts/sf-distant-galaxy-font/SfDistantGalaxy-0l3d.ttf"
            text_color = (255, 255, 255, 255) # White, fully opaque RGBA

            dynamic_fontsize = int(target_height * 0.055)
            try:
                title_font = ImageFont.truetype(font_path, dynamic_fontsize)
            except IOError:
                print(f"Warning: Could not load font '{font_path}' for title. Falling back to default.")
                title_font = ImageFont.load_default()

            max_title_line_width = int(target_width * 0.85)
            title_lines = dynamic_wrap_text(article_title, title_font, max_title_line_width)

            current_y_for_text = target_height * 0.02
            total_title_height = 0

            for line in title_lines:
                try:
                    bbox = draw.textbbox((0,0), line, font=title_font)
                    line_width = bbox[2] - bbox[0]
                    line_height = bbox[3] - bbox[1]
                except AttributeError:
                    line_width, line_height = draw.textsize(line, font=title_font)

                x_pos = (img_pil.width - line_width) / 2
                
                draw.text((x_pos, current_y_for_text), line, font=title_font, fill=text_color)
                
                current_y_for_text += line_height + int(target_height * 0.005)
                total_title_height += line_height + int(target_height * 0.005)

            # --- IMAGE OVERLAY LOGIC ---
            image_bottom_y = current_y_for_text
            
            if downloaded_image_paths:
                image_index = int(t_in_clip / image_duration_per_clip) % len(downloaded_image_paths)
                current_image_path = downloaded_image_paths[image_index]

                try:
                    img_pil_overlay = Image.open(current_image_path).convert("RGBA")

                    target_image_max_dim = int(target_width * 0.96)

                    orig_img_w, orig_img_h = img_pil_overlay.size

                    scale_w = target_image_max_dim / orig_img_w
                    scale_h = target_image_max_dim / orig_img_h
                    scale = min(scale_w, scale_h)

                    resized_img_w = int(orig_img_w * scale)
                    resized_img_h = int(orig_img_h * scale)
                    img_pil_overlay = img_pil_overlay.resize((resized_img_w, resized_img_h), Image.LANCZOS)

                    image_x_pos = (target_width - resized_img_w) / 2
                    image_y_pos = int((target_height * 0.02) + total_title_height + target_height * 0.03)
                    
                    # Paste the RGBA overlay onto the RGBA base image, using the overlay's alpha as a mask
                    img_pil.paste(img_pil_overlay, (int(image_x_pos), int(image_y_pos)), img_pil_overlay)
                    
                    image_bottom_y = image_y_pos + resized_img_h + int(target_height * 0.03)

                except Exception as img_e:
                    print(f"Error overlaying image {current_image_path} at time {t_in_clip:.2f}s: {img_e}")

            # --- SUBTITLE DRAWING LOGIC ---
            # Using same font size and style as title for consistency, but positioned at bottom
            subtitle_font_size = int(target_height * 0.05) # Slightly smaller than title for readability
            try:
                subtitle_font = ImageFont.truetype(font_path, subtitle_font_size)
            except IOError:
                print(f"Warning: Could not load font '{font_path}' for subtitle. Falling back to default.")
                subtitle_font = ImageFont.load_default()
            
            # Subtitle color: Bright yellow, fully opaque RGBA
            subtitle_fill_color = (255, 255, 0, 255) # Yellow, fully opaque RGBA

            current_subtitle_text = ""
            for sub in subtitles:
                if sub['start'] <= t_actual_transcript < sub['end']:
                    current_subtitle_text = sub['text']
                    if int(t_in_clip * 10) % 10 == 0:
                        print(f"    Subtitle found: \"{current_subtitle_text}\" (SRT range: {sub['start']:.2f}-{sub['end']:.2f})")
                    break
            
            if not current_subtitle_text and int(t_in_clip * 10) % 10 == 0:
                 print(f"    No subtitle found for actual transcript time {t_actual_transcript:.2f}s.")

            if current_subtitle_text:
                max_subtitle_line_width = int(target_width * 0.9)
                subtitle_lines = dynamic_wrap_text(current_subtitle_text, subtitle_font, max_subtitle_line_width)

                total_subtitle_height = 0
                dummy_draw = ImageDraw.Draw(Image.new("RGBA", (1,1))) # Use RGBA for dummy
                for line in subtitle_lines:
                     try:
                        bbox = dummy_draw.textbbox((0,0), line, font=subtitle_font)
                        total_subtitle_height += (bbox[3] - bbox[1]) + int(target_height * 0.005)
                     except AttributeError:
                        _, line_h = dummy_draw.textsize(line, font=subtitle_font)
                        total_subtitle_height += line_h + int(target_height * 0.005)

                target_bottom_margin = int(target_height * 0.18)
                # Calculate the Y position for the bottom of the lowest subtitle line
                desired_y_for_bottom_of_subtitles = target_height - target_bottom_margin
                
                # Calculate the starting Y position for the *first* subtitle line
                current_y_for_subtitle = desired_y_for_bottom_of_subtitles - total_subtitle_height

                # Ensure subtitles don't overlap with images/title if they are too long
                subtitle_y_start_after_elements = image_bottom_y + int(target_height * 0.02) # Add a small buffer below image
                if current_y_for_subtitle < subtitle_y_start_after_elements:
                    current_y_for_subtitle = subtitle_y_start_after_elements
                    print(f"      Adjusted subtitle start Y to {current_y_for_subtitle:.0f} to avoid overlap.")

                for line in subtitle_lines:
                    try:
                        bbox = draw.textbbox((0,0), line, font=subtitle_font)
                        line_width = bbox[2] - bbox[0]
                        line_height = bbox[3] - bbox[1]
                    except AttributeError:
                        line_width, line_height = draw.textsize(line, font=subtitle_font)

                    x_pos = (img_pil.width - line_width) / 2
                    
                    if int(t_in_clip * 10) % 10 == 0:
                        print(f"      Drawing subtitle: '{line}' at X={x_pos}, Y={current_y_for_subtitle}, W={line_width}, H={line_height}, Color={subtitle_fill_color}")
                    
                    # Draw a solid black rectangle behind the subtitle for better contrast,
                    # ensuring it's opaque and drawn *before* the text.
                    padding_x = int(target_width * 0.01) # Small padding around text
                    padding_y = int(target_height * 0.005)
                    background_box_x1 = x_pos - padding_x
                    background_box_y1 = current_y_for_subtitle - padding_y
                    background_box_x2 = x_pos + line_width + padding_x
                    background_box_y2 = current_y_for_subtitle + line_height + padding_y
                    
                    # Ensure coordinates are within bounds
                    background_box_x1 = max(0, background_box_x1)
                    background_box_y1 = max(0, background_box_y1)
                    background_box_x2 = min(target_width, background_box_x2)
                    background_box_y2 = min(target_height, background_box_y2)

                    draw.rectangle([background_box_x1, background_box_y1, background_box_x2, background_box_y2],
                                   fill=(0, 0, 0, 180)) # Semi-transparent black background

                    draw.text((x_pos, current_y_for_subtitle), line, font=subtitle_font, fill=subtitle_fill_color)
                    current_y_for_subtitle += line_height + int(target_height * 0.005)
            
            # Convert back to RGB before returning to MoviePy
            return np.array(img_pil.convert("RGB"))

        final_clip_with_elements = final_clip.fl(draw_elements_on_frame)

        sanitized_category_folder = re.sub(r'[\\/*?:"<>|]', "", category_folder).strip()
        if not sanitized_category_folder:
            sanitized_category_folder = "Uncategorized_Shorts"

        output_category_dir = os.path.join(output_base_dir, sanitized_category_folder)
        os.makedirs(output_category_dir, exist_ok=True)

        sanitized_title = re.sub(r'[\\/*?:"<>\'"]', "", article_title.replace('/', '_'))
        output_filepath = os.path.join(output_category_dir, f"{sanitized_title}_short.mp4")

        print(f"Writing final video to: {output_filepath}")
        final_clip_with_elements.write_videofile(
            output_filepath, 
            codec="libx264", 
            audio_codec="libmp3lame", 
            fps=30, 
            preset="medium",
            threads=os.cpu_count(),
            verbose=True, 
            logger='bar'
        )

        audio_clip.close()
        main_video_clip.close()
        video_clip.close() 
        cropped_clip.close() 
        final_clip.close()

        print(f"Successfully created short for '{article_title}'.")
        return True

    except Exception as e:
        print(f"Error creating short for '{article_title}': {e}")
        import traceback
        traceback.print_exc() # Print full traceback for better debugging
        try:
            if 'audio_clip' in locals() and audio_clip: audio_clip.close()
            if 'main_video_clip' in locals() and main_video_clip: main_video_clip.close()
            if 'video_clip' in locals() and video_clip: video_clip.close()
            if 'cropped_clip' in locals() and cropped_clip: cropped_clip.close()
            if 'final_clip' in locals() and final_clip: final_clip.close()
        except Exception as close_e:
            print(f"Error closing clips: {close_e}")
        return False
    finally:
        clean_temp_images(TEMP_IMAGE_DIRECTORY)

# --- Main Execution Logic ---
if __name__ == "__main__":
    os.makedirs(GENERATED_SHORTS_DIRECTORY, exist_ok=True)
    os.makedirs(TEMP_IMAGE_DIRECTORY, exist_ok=True)

    processed_shorts_titles = get_processed_shorts_titles(TRACKING_SHORTS_FILE)
    print(f"Loaded {len(processed_shorts_titles)} previously created shorts titles.")

    for root, dirs, files in os.walk(GENERATED_AUDIO_DIRECTORY):
        category_folder = os.path.basename(root)

        for filename in files:
            if filename.endswith(".mp3"):
                audio_filepath = os.path.join(root, filename)
                
                article_title = os.path.splitext(filename)[0]

                if article_title in processed_shorts_titles:
                    print(f"\n'{article_title}' has already had a short created. Skipping.")
                    continue

                print(f"\nProcessing audio file: {audio_filepath} for short creation.")
                
                article_content = ""
                article_text_filepath = os.path.join(GENERATED_ARTICLES_DIRECTORY, category_folder, f"{article_title}.txt")
                if os.path.exists(article_text_filepath):
                    with open(article_text_filepath, 'r', encoding='utf-8') as f:
                        article_content = f.read()
                    print(f"Loaded article content from: {article_text_filepath}")
                else:
                    print(f"Warning: No article text file found for '{article_title}' at {article_text_filepath}. No images will be extracted.")
                
                image_urls = extract_image_urls_from_article(article_content)
                if image_urls:
                    print(f"Found {len(image_urls)} image URLs in article.")
                else:
                    print("No image URLs found in article content.")

                srt_filepath = os.path.join(GENERATED_TRANSCRIPT_DIRECTORY, category_folder, f"{article_title}.srt")

                short_created = create_youtube_short(
                    audio_filepath,
                    MINECRAFT_FOOTAGE_PATH,
                    GENERATED_SHORTS_DIRECTORY,
                    article_title,
                    category_folder,
                    image_urls,
                    srt_filepath 
                )

                if short_created:
                    log_processed_short_title(article_title, TRACKING_SHORTS_FILE)
                    print(f"Short creation and logging complete for '{article_title}'.")
                else:
                    print(f"Short creation failed for '{article_title}'. Not logging.")
    
    print("\n--- All short creations complete. ---")
    clean_temp_images(TEMP_IMAGE_DIRECTORY)

Loaded 10 previously created shorts titles.

'Mulita' has already had a short created. Skipping.

'Antrech Arms' has already had a short created. Skipping.

'Mission to Batuu' has already had a short created. Skipping.

'Eos Morne' has already had a short created. Skipping.

Processing audio file: ../data/generated_audio\Character stubs\Unidentified Second Prince.mp3 for short creation.
Loaded article content from: ../data/generated_articles\Character stubs\Unidentified Second Prince.txt
No image URLs found in article content.
Cleaned temporary image directory: ../data/temp_images
Loaded 17 subtitle entries from: ../data/generated_transcripts\Character stubs\Unidentified Second Prince.srt
Loaded audio: Unidentified Second Prince.mp3 (Duration: 26.14s)
Loaded main video: minecraft01.mp4 (Duration: 1076.82s)
Selected video segment: 680.06s to 706.20s (Clip duration: 26.14s)


t:  32%|███▏      | 566/1769 [4:09:13<02:59,  6.70it/s, now=None]

Cropping width from 1920 to 607 to fit 9:16.
Resized to final resolution: 1080x1920
No image URLs found for overlay.
  Frame at t_in_clip: 0.00s (Time for subtitle lookup: 0.00s)
    Subtitle found: "IN THE EMPIRES TWILIGHT YEARS" (SRT range: 0.00-1.78)
      Drawing subtitle: 'IN THE EMPIRES' at X=118.0, Y=1429, W=844, H=64, Color=(255, 255, 0, 255)
      Drawing subtitle: 'TWILIGHT YEARS' at X=86.0, Y=1502, W=908, H=64, Color=(255, 255, 0, 255)
Writing final video to: ../data/generated_shorts\Character stubs\Unidentified Second Prince_short.mp4
Moviepy - Building video ../data/generated_shorts\Character stubs\Unidentified Second Prince_short.mp4.
MoviePy - Writing audio in Unidentified Second Prince_shortTEMP_MPY_wvf_snd.mp3


t:  32%|███▏      | 566/1769 [4:09:13<02:59,  6.70it/s, now=None]

MoviePy - Done.
Moviepy - Writing video ../data/generated_shorts\Character stubs\Unidentified Second Prince_short.mp4



...

  Frame at t_in_clip: 22.00s (Time for subtitle lookup: 22.00s)
    No subtitle found for actual transcript time 22.00s.
  Frame at t_in_clip: 22.03s (Time for subtitle lookup: 22.03s)
    No subtitle found for actual transcript time 22.03s.


t:  97%|█████████▋| 663/681 [01:28<00:02,  7.22it/s, now=None]

  Frame at t_in_clip: 22.07s (Time for subtitle lookup: 22.07s)
    No subtitle found for actual transcript time 22.07s.


t:  32%|███▏      | 566/1769 [4:15:14<02:59,  6.70it/s, now=None]

Moviepy - Done !
Moviepy - video ready ../data/generated_shorts\Science stubs\Kilogram_short.mp4
Successfully created short for 'Kilogram'.
Cleaned temporary image directory: ../data/temp_images
Short creation and logging complete for 'Kilogram'.

'What Mamma Doesn't Know' has already had a short created. Skipping.

--- All short creations complete. ---
Cleaned temporary image directory: ../data/temp_images


In [22]:
pip install google-api-python-client google-auth-oauthlib google-auth-httplib2

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import os
import glob
import pickle
import datetime
import logging
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from googleapiclient.errors import HttpError
from dotenv import load_dotenv

# --- Load environment variables ---
load_dotenv()

# --- Configuration ---
CLIENT_SECRETS_FILE = r'C:\egyetem\github-ml\holocron-generator\client_secret_12090445080-o5mvqmklk0ac7uu7q4co9hib5sgi01fn.apps.googleusercontent.com.json'
UPLOADED_LOG_FILE = '../data/uploaded_shorts.log'
BASE_VIDEO_DIR = '../data/generated_shorts'
BASE_TEXT_DIR = '../data/generated_text'

# YouTube API scopes - 'youtube.upload' is essential for uploading
SCOPES = ['https://www.googleapis.com/auth/youtube.upload']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

# General hashtags to append to all descriptions
GLOBAL_HASHTAGS = [
    '#shorts',
    '#starwars',
    '#anakin',
    '#legends',
    '#starwarsfans', 
    '#jedi',
    '#sith',
    '#lightsaber',
    '#starwarslegends',
    '#starwarsfan',
    '#starwarscommunity',
    '#yoda',
    '#darthvader',
    '#kenobi',
    '#starwarsreels',
    '#force'
]

# --- Logging Setup ---
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    handlers=[
                        logging.FileHandler("uploader.log"), # Log to a file
                        logging.StreamHandler() # Also log to console
                    ])

# --- OAuth Authentication Function ---
def get_authenticated_service():
    """Authenticates with Google OAuth and returns a YouTube API service."""
    credentials = None
    # Load credentials from token.pickle if it exists
    if os.path.exists('token.pickle'):
        try:
            with open('token.pickle', 'rb') as token:
                credentials = pickle.load(token)
            logging.info("Loaded credentials from token.pickle")
        except Exception as e:
            logging.warning(f"Error loading token.pickle: {e}. Will re-authenticate.")
            credentials = None

    # If no valid credentials, initiate the OAuth flow
    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            logging.info("Access token expired, attempting to refresh...")
            try:
                credentials.refresh(Request())
                logging.info("Access token refreshed successfully.")
            except Exception as e:
                logging.error(f"Error refreshing token: {e}. Re-initiating full OAuth flow.")
                credentials = None # Force full re-authentication
        else:
            logging.info("No valid credentials, initiating new OAuth flow...")
            try:
                flow = InstalledAppFlow.from_client_secrets_file(
                    CLIENT_SECRETS_FILE, SCOPES)
                # This will open a browser for the user to authorize
                credentials = flow.run_local_server(port=0)
                logging.info("OAuth flow completed successfully.")
            except Exception as e:
                logging.error(f"Failed to complete OAuth flow: {e}")
                raise # Re-raise the exception to stop execution

        # Save the new/refreshed credentials
        try:
            with open('token.pickle', 'wb') as token:
                pickle.dump(credentials, token)
            logging.info("Credentials saved to token.pickle")
        except Exception as e:
            logging.error(f"Failed to save credentials to token.pickle: {e}")
            # Non-fatal, but user might need to re-auth more often

    return build(API_SERVICE_NAME, API_VERSION, credentials=credentials)

# --- Video Upload Function ---
def upload_video(youtube_service, video_path, title, description, tags, category_id='24', privacy_status='public'):
    """
    Uploads a video to YouTube.

    Args:
        youtube_service: An authenticated YouTube API service object.
        video_path (str): Path to the video file.
        title (str): Title of the video.
        description (str): Description of the video.
        tags (list): List of tags for the video.
        category_id (str): YouTube category ID (default '22' for People & Blogs).
        privacy_status (str): 'public', 'private', or 'unlisted'.

    Returns:
        str: The YouTube video ID if successful, None otherwise.
    """
    body = {
        'snippet': {
            'title': title,
            'description': description,
            'tags': tags,
            'categoryId': category_id
        },
        'status': {
            'privacyStatus': privacy_status
        },
        'recordingDetails': { # Optional: for Shorts, you might want to specify this
            'recordingDate': datetime.datetime.now(datetime.timezone.utc).isoformat()
        }
    }

    media_body = MediaFileUpload(video_path, chunksize=-1, resumable=True)

    try:
        request = youtube_service.videos().insert(
            part='snippet,status,recordingDetails',
            body=body,
            media_body=media_body
        )
        response = None
        while response is None:
            status, response = request.next_chunk()
            if status:
                logging.info(f"Uploaded {int(status.resumable_progress * 100)}% of {os.path.basename(video_path)}")

        if 'id' in response:
            logging.info(f"Successfully uploaded: '{title}' (Video ID: {response['id']})")
            return response['id']
        else:
            logging.error(f"Upload failed for {os.path.basename(video_path)} with unexpected response: {response}")
            return None

    except HttpError as e:
        logging.error(f"An HTTP error occurred during upload for {os.path.basename(video_path)}: {e}")
        return None
    except Exception as e:
        logging.error(f"An unexpected error occurred during upload for {os.path.basename(video_path)}: {e}")
        return None

# --- File Management and Logging ---
def get_uploaded_videos():
    """Reads the log file and returns a set of already uploaded video file paths."""
    uploaded_files = set()
    if os.path.exists(UPLOADED_LOG_FILE):
        with open(UPLOADED_LOG_FILE, 'r', encoding='utf-8') as f:
            for line in f:
                parts = line.strip().split(' | ')
                if len(parts) >= 2:
                    # Get the path from the log file
                    logged_path = parts[1]

                    # --- NEW CHANGE HERE ---
                    # Normalize the logged path to match os.path.relpath's output
                    # This often means converting '/' to '\' on Windows or vice versa.
                    # os.path.normpath() is good for this, it handles / and \ consistently.
                    normalized_logged_path = os.path.normpath(logged_path)

                    uploaded_files.add(normalized_logged_path)
                    # --- END OF NEW CHANGE ---

    return uploaded_files

def log_uploaded_video(video_path, video_id, title):
    """Appends the details of an uploaded video to the log file."""
    timestamp = datetime.datetime.now().isoformat()
    # It's good practice to normalize path for logging as well, to ensure consistency
    # with how it's read back. os.path.normpath will use native separators.
    normalized_video_path = os.path.normpath(video_path)
    with open(UPLOADED_LOG_FILE, 'a', encoding='utf-8') as f:
        f.write(f"{timestamp} | {normalized_video_path} | {video_id} | {title}\n")
    logging.info(f"Logged upload: {os.path.basename(video_path)}")

# --- Main Logic ---
def main():
    try:
        youtube = get_authenticated_service()
    except Exception as e:
        logging.critical(f"Failed to authenticate. Exiting: {e}")
        return

    uploaded_videos = get_uploaded_videos()
    logging.info(f"Found {len(uploaded_videos)} already uploaded videos in log.")

    # Iterate through categories (subdirectories in BASE_VIDEO_DIR)
    for category_dir in os.listdir(BASE_VIDEO_DIR):
        category_video_path = os.path.join(BASE_VIDEO_DIR, category_dir)
        # --- IMPORTANT: Adjust category_text_path based on how it's relative to main_uploader.py ---
        # If BASE_TEXT_DIR is already relative to the script's execution directory
        # e.g., 'data/generated_text' in the project root
        category_text_path = os.path.join(BASE_TEXT_DIR, category_dir)
        # If your 'data' folder is one level up from your script
        # script_dir = os.path.dirname(__file__)
        # category_text_path = os.path.join(script_dir, '..', BASE_TEXT_DIR, category_dir) # Example: if script in 'scripts/' and data in project root

        if not os.path.isdir(category_video_path):
            continue # Skip if it's not a directory

        logging.info(f"Processing category: {category_dir}")

        # Find all MP4 files in the current category
        for video_file in glob.glob(os.path.join(category_video_path, '*.mp4')):
            relative_video_path = os.path.relpath(video_file) # Store relative path for consistency in log

            if relative_video_path in uploaded_videos:
                logging.info(f"Skipping already uploaded video: {os.path.basename(video_file)}")
                continue

            # Determine corresponding text file path
            video_basename_with_ext = os.path.basename(video_file)
            video_name_without_ext_full = os.path.splitext(video_basename_with_ext)[0] # e.g., "Mission to Batuu_short"

            # Remove the "_short" suffix if present to get the base name for text file
            if video_name_without_ext_full.endswith('_short'):
                video_name_for_text_file = video_name_without_ext_full.removesuffix('_short')
            else:
                video_name_for_text_file = video_name_without_ext_full

            # *** --- NEW CHANGE: Append "_rephrased.txt" for text file path --- ***
            text_file_path_for_video = os.path.join(category_text_path, f"{video_name_for_text_file}_rephrased.txt")

            if not os.path.exists(text_file_path_for_video): # Use the newly constructed path here
                logging.warning(f"No description file found for {os.path.basename(video_file)} ({text_file_path_for_video}). Skipping.")
                continue

            # Read description
            with open(text_file_path_for_video, 'r', encoding='utf-8') as f: # Use the newly constructed path here
                description_content = f.read().strip()

            # Construct full description with hashtags
            full_description = f"{description_content}\n\n" + " ".join(GLOBAL_HASHTAGS)

            # Use the clean name for the video title (still without _short or _rephrased)
            video_title = video_name_for_text_file.replace('_', ' ').title() # Basic title formatting

            # Prepare tags (example: based on categories and global hashtags)
            tags = [category_dir.replace('_', ' ').lower()] + [h.strip('#') for h in GLOBAL_HASHTAGS]
            tags = list(set(tags)) # Remove duplicates

            logging.info(f"Attempting to upload: {video_title}")
            logging.info(f"Description: {full_description[:100]}...") # Show first 100 chars
            logging.info(f"Tags: {', '.join(tags)}")

            # Perform the upload
            video_id = upload_video(youtube, video_file, video_title, full_description, tags)

            if video_id:
                log_uploaded_video(relative_video_path, video_id, video_title)
            else:
                logging.error(f"Failed to upload or log: {os.path.basename(video_file)}")

if __name__ == '__main__':
    main()

2025-07-29 13:11:53,703 - INFO - Loaded credentials from token.pickle


2025-07-29 13:11:53,708 - INFO - file_cache is only supported with oauth2client<4.0.0
2025-07-29 13:11:53,726 - INFO - Found 10 already uploaded videos in log.
2025-07-29 13:11:53,728 - INFO - Processing category: Arboreal planets
2025-07-29 13:11:53,730 - INFO - Attempting to upload: Mulita
2025-07-29 13:11:53,731 - INFO - Description: Deep within Wild Space lies Mulita, a forested planet teeming with the Drengir, sentient carnivorous...
2025-07-29 13:11:53,732 - INFO - Tags: lightsaber, darthvader, starwarscommunity, starwarsfans, starwarslegends, shorts, starwarsfan, arboreal planets, legends, anakin, starwars, jedi, kenobi, starwarsreels, sith, force, yoda
2025-07-29 13:11:54,552 - ERROR - An HTTP error occurred during upload for Mulita_short.mp4: <HttpError 400 when requesting None returned "The user has exceeded the number of videos they may upload.". Details: "[{'message': 'The user has exceeded the number of videos they may upload.', 'domain': 'youtube.video', 'reason': 'upload

In [8]:
import os
import glob
import pickle
import datetime
import logging
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from googleapiclient.errors import HttpError
from dotenv import load_dotenv

# --- Load environment variables ---
load_dotenv()

# --- Configuration ---
CLIENT_SECRETS_FILE = r'C:\egyetem\github-ml\holocron-generator\client_secret_12090445080-o5mvqmklk0ac7uu7q4co9hib5sgi01fn.apps.googleusercontent.com.json'
UPLOADED_LOG_FILE = '../data/uploaded_shorts.log'
BASE_VIDEO_DIR = '../data/generated_shorts'
BASE_TEXT_DIR = '../data/generated_text'
TOKEN_PICKLE_FILE = 'token.pickle'

# YouTube API scopes - 'youtube.upload' is essential for uploading
SCOPES = ['https://www.googleapis.com/auth/youtube.upload']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

# General hashtags to append to all descriptions
GLOBAL_HASHTAGS = [
    '#shorts',
    '#starwars',
    '#anakin',
    '#legends',
    '#starwarsfans', 
    '#jedi',
    '#sith',
    '#lightsaber',
    '#starwarslegends',
    '#starwarsfan',
    '#starwarscommunity',
    '#yoda',
    '#darthvader',
    '#kenobi',
    '#starwarsreels',
    '#force'
]

# --- Logging Setup ---
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    handlers=[
                        logging.FileHandler("uploader.log"), # Log to a file
                        logging.StreamHandler() # Also log to console
                    ])

# --- OAuth Authentication Function (No functional change) ---
def get_authenticated_service():
    """Authenticates with Google OAuth and returns a YouTube API service."""
    credentials = None
    if os.path.exists(TOKEN_PICKLE_FILE):
        try:
            with open(TOKEN_PICKLE_FILE, 'rb') as token:
                credentials = pickle.load(token)
            logging.info("Loaded credentials from token.pickle")
        except Exception as e:
            logging.warning(f"Error loading token.pickle: {e}. Will re-authenticate.")
            credentials = None

    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            logging.info("Access token expired, attempting to refresh...")
            try:
                credentials.refresh(Request())
                logging.info("Access token refreshed successfully.")
            except Exception as e:
                logging.error(f"Error refreshing token: {e}. Re-initiating full OAuth flow.")
                credentials = None
        else:
            logging.info("No valid credentials, initiating new OAuth flow...")
            try:
                flow = InstalledAppFlow.from_client_secrets_file(
                    CLIENT_SECRETS_FILE, SCOPES)
                credentials = flow.run_local_server(port=0)
                logging.info("OAuth flow completed successfully.")
            except Exception as e:
                logging.error(f"Failed to complete OAuth flow: {e}")
                raise

        try:
            with open(TOKEN_PICKLE_FILE, 'wb') as token:
                pickle.dump(credentials, token)
            logging.info("Credentials saved to token.pickle")
        except Exception as e:
            logging.error(f"Failed to save credentials to token.pickle: {e}")

    return build(API_SERVICE_NAME, API_VERSION, credentials=credentials)

# --- Video Upload Function (No functional change) ---
def upload_video(youtube_service, video_path, title, description, tags, category_id='24', privacy_status='public'):
    """
    Uploads a video to YouTube.
    ... (rest of the upload_video function remains the same) ...
    """
    body = {
        'snippet': {
            'title': title,
            'description': description,
            'tags': tags,
            'categoryId': category_id
        },
        'status': {
            'privacyStatus': privacy_status
        },
        'recordingDetails': {
            'recordingDate': datetime.datetime.now(datetime.timezone.utc).isoformat()
        }
    }

    media_body = MediaFileUpload(video_path, chunksize=-1, resumable=True)

    try:
        request = youtube_service.videos().insert(
            part='snippet,status,recordingDetails',
            body=body,
            media_body=media_body
        )
        response = None
        while response is None:
            status, response = request.next_chunk()
            if status:
                logging.info(f"Uploaded {int(status.resumable_progress * 100)}% of {os.path.basename(video_path)}")

        if 'id' in response:
            logging.info(f"Successfully uploaded: '{title}' (Video ID: {response['id']})")
            return response['id']
        else:
            logging.error(f"Upload failed for {os.path.basename(video_path)} with unexpected response: {response}")
            return None

    except HttpError as e:
        logging.error(f"An HTTP error occurred during upload for {os.path.basename(video_path)}: {e}")
        return None
    except Exception as e:
        logging.error(f"An unexpected error occurred during upload for {os.path.basename(video_path)}: {e}")
        return None

# --- File Management and Logging (SIMPLIFIED TO TITLE ONLY) ---
def get_uploaded_videos():
    """Reads the log file and returns a set of already uploaded video titles."""
    uploaded_titles = set()
    if os.path.exists(UPLOADED_LOG_FILE):
        with open(UPLOADED_LOG_FILE, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if line: # Ensure line is not empty
                    uploaded_titles.add(line) # Add the entire line as the title
    return uploaded_titles

def log_uploaded_video(title):
    """Appends the title of an uploaded video to the log file."""
    with open(UPLOADED_LOG_FILE, 'a', encoding='utf-8') as f:
        f.write(f"{title}\n") # Just write the title
    logging.info(f"Logged upload: '{title}'")

# --- Main Logic (ADJUSTED FOR TITLE MATCHING AND DERIVATION) ---
def main():
    try:
        youtube = get_authenticated_service()
    except Exception as e:
        logging.critical(f"Failed to authenticate. Exiting: {e}")
        return

    uploaded_titles = get_uploaded_videos()
    logging.info(f"Found {len(uploaded_titles)} already uploaded videos (by title) in log.")

    for category_dir in os.listdir(BASE_VIDEO_DIR):
        category_video_path = os.path.join(BASE_VIDEO_DIR, category_dir)
        category_text_path = os.path.join(BASE_TEXT_DIR, category_dir)

        if not os.path.isdir(category_video_path):
            continue

        logging.info(f"Processing category: {category_dir}")

        for video_file in glob.glob(os.path.join(category_video_path, '*.mp4')):
            video_basename_with_ext = os.path.basename(video_file)
            video_name_without_ext_full = os.path.splitext(video_basename_with_ext)[0]

            if video_name_without_ext_full.endswith('_short'):
                video_name_base = video_name_without_ext_full.removesuffix('_short')
            else:
                video_name_base = video_name_without_ext_full

            # --- DYNAMICALLY DETERMINE FINAL VIDEO TITLE ---
            # This logic must accurately convert video_name_base to your desired YouTube title.
            # Start with a default transformation
            video_title = video_name_base.replace('_', ' ').title()

            # Apply specific overrides/corrections based on your examples
            # This order matters if patterns overlap!

            # Example: "Kovor system_Legends" -> "Kovor system/Legends"
            if "Kovor system_Legends" == video_name_base: # Use exact match for specific cases
                video_title = "Kovor system/Legends"
            
            # Example: "What Mamma Doesnt Know" -> "What Mamma Doesn't Know"
            if "What Mamma Doesnt Know" == video_name_base:
                 video_title = "What Mamma Doesn't Know"
            
            # Example: "Iguana (speeder bike)"
            if "Iguana (speeder bike)" == video_name_base:
                video_title = "Iguana (speeder bike)" # If the base name is already perfect, use it directly
                                                      # The .title() on this would make it "Iguana (Speeder Bike)"
                                                      # which might not be desired.

            # If you have more specific patterns like "Protocol 514" or "Unidentified Second Prince"
            # that are already perfect in video_name_base, you can add similar if-statements
            # if video_name_base needs to override the default .title() conversion.
            # Example:
            # if "Protocol 514" == video_name_base:
            #     video_title = "Protocol 514"
            # if "Unidentified Second Prince" == video_name_base:
            #     video_title = "Unidentified Second Prince"

            # --- CRITICAL: Check if this derived video_title is already in our log ---
            if video_title in uploaded_titles:
                logging.info(f"Skipping already uploaded video (title found in log): '{video_title}'")
                continue

            # --- Determine corresponding text file path (uses video_name_base) ---
            text_file_path_for_video = os.path.join(category_text_path, f"{video_name_base}_rephrased.txt")

            if not os.path.exists(text_file_path_for_video):
                logging.warning(f"No description file found for {os.path.basename(video_file)} ({text_file_path_for_video}). Skipping.")
                continue

            with open(text_file_path_for_video, 'r', encoding='utf-8') as f:
                description_content = f.read().strip()

            full_description = f"{description_content}\n\n" + " ".join(GLOBAL_HASHTAGS)

            tags = [category_dir.replace('_', ' ').lower()] + [h.strip('#') for h in GLOBAL_HASHTAGS]
            tags = list(set(tags))

            logging.info(f"Attempting to upload: '{video_title}'")
            logging.info(f"Description: {full_description[:100]}...")
            logging.info(f"Tags: {', '.join(tags)}")

            video_id = upload_video(youtube, video_file, video_title, full_description, tags)

            if video_id:
                log_uploaded_video(video_title) # Log ONLY the title
            else:
                logging.error(f"Failed to upload: '{video_title}'. Will retry on next run unless manually logged.")

if __name__ == '__main__':
    main()

2025-07-29 13:22:16,091 - INFO - Loaded credentials from token.pickle
2025-07-29 13:22:16,095 - INFO - file_cache is only supported with oauth2client<4.0.0
2025-07-29 13:22:16,102 - INFO - Found 10 already uploaded videos (by title) in log.
2025-07-29 13:22:16,104 - INFO - Processing category: Arboreal planets
2025-07-29 13:22:16,106 - INFO - Skipping already uploaded video (title found in log): 'Mulita'
2025-07-29 13:22:16,108 - INFO - Processing category: Arms manufacturing companies
2025-07-29 13:22:16,111 - INFO - Skipping already uploaded video (title found in log): 'Antrech Arms'
2025-07-29 13:22:16,112 - INFO - Processing category: Battles in 19 BBY
2025-07-29 13:22:16,113 - INFO - Attempting to upload: 'Mission To Batuu'
2025-07-29 13:22:16,116 - INFO - Description: During the Clone Wars, a Separatist droid factory on Mokivj prompted a mission led by Anakin Skywalk...
2025-07-29 13:22:16,118 - INFO - Tags: lightsaber, darthvader, starwarscommunity, starwarsfans, starwarslegends